# How to write macros
<br>
<br>
<img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" />
Sardana-Training by ALBA Synchrotron is licensed under the Creative Commons Attribution 4.0 International License.  
To view a copy of this license, visit http://creativecommons.org/licenses/by/4.0/.

# Macro function vs. macro class

## Function
* For programmers with background on a functional language (e.g. Fortran, Matlab, SPEC)
* Sequencial procedures to run an experiment

## Class
* For proammers with background on object-oriented languages (e.g. Java, C++, C#)
* Overriding an existing macros (e.g. inherit from `mv` macro and extend its funcionality)
* Some advanced features require use of classes e.g. define hook places or mandatory environment variables

# Macro plugins discovery
* Sardana built-in macros are importable from `sardana.macroserver.macros`
* The plugin discovery system is based on directory scanning and python module inspection
* Custom macros should be installed in one of the `MacroPath` directories:
 * Create `macros` directory: `mkdir /home/vagrant/macros`
 * In spock: `_MACRO_SERVER.put_property({"MacroPath":["/home/vagrant/macros", "/home/vagrant/.local/lib/python3.7/site-packages/sardana/macroserver/macros/examples"]})`
 * Restart the Sardana server
* The path order is important! Macros in the higher position paths will take precedence over the lower position paths.

# Macro development tools

* `edmac <macro_name> [<module_name>]`
* `relmac`, `relmaclib`, `addmaclib`, `rellib`
* `www`, `post_mortem`
* `debug on|off`

# Hello world

* In spock type: `edmac hello_world mymacros`
* Implement a function decorated with `macro` decorator that can be found in `sardana.macroserver.macro`
* Use `self.output()` to print something...
* Exit from editor and apply changes on the server [y]
* Run `hello_world` macro

In [2]:
from sardana.macroserver.macro import Macro, macro, Type

@macro()
def hello_world(self):
    """Macro hello_world"""
    self.output("Running hello_world...")

# MACRO FEATURES

### Logging in macros

* Macros may log messages to various destinations
 * Spock or other clients
 * Macro logging/reporting feature
 * MacroServer log files
* Different log levels are possible e.g. output, error, debug

### Logging in macros - demo

* Edit `move` macro and instead of `self.output` use `self.info`
* `wa` (table view)
* `umv mot01 10` (update block view)

### Macro parameters

* Benefits
 * The parameters validation prevents runtime errors
 * Spock offers autocompletion facilities
 * Autogenerated documentation

### Macro parameters

* Parameters are defined either as arguments of the `macro` decorator for the macro function or as `param_def` class member for the macro class
* A parameter is characterized by: name, type, default value and description
* Parameter values arrives as positional arguments to the macro function or to the run method of the macro class

### Macro parameters - development

* Open `/home/vagrant/mymacros.py` module with an external editor
* Develop a new macro: `move`
* Add two macro parameters according to the [Adding macro parameters documentation](https://sardana-controls.org/devel/howto_macros/macros_general.html#adding-parameters-to-your-macro)
  * moveable of type `Moveable`
  * position (to move) of type `Float`
* Move the motor to the position according to the [Moveable API](https://sardana-controls.org/devel/api/sardana/taurus/core/tango/sardana/pool.html) (`moveable.move(position)`)
* Apply the new code on the server using the `relmaclib mymacros` macro (in Spock)
* Continue developing your macro to print the motor position (`moveable.getPosition()`)
* Apply the new code on the server using the `relmaclib mymacros` macro (in Spock)
* Run `move mot01 10`

In [1]:
from sardana.macroserver.macro import macro, Type

@macro([["moveable", Type.Moveable, None, "moveable to move"],
        ["position", Type.Float, None, "absolute position"]])
def move(self, moveable, position):
    """This macro moves a moveable to the specified position"""
    moveable.move(position)
    self.output("{} is now at {}", moveable.getName(), moveable.getPosition())

AttributeError: 'TypeNames' object has no attribute 'Moveable'

### Macro parameters - repeat parameters

* Allows to pass as parameter value a list of repetitions of the repeat parameter member(s)
* Repeat parameters allow to:
 * restrict the minimum and/or maximum number of repetitions
 * nest repeat parameters inside of another repeat parameters
 * define multiple repeat parameters in the same macro


### Macro parameters - repeat parameters - demo

* `prdef pt4`
* `prdef pt5`

### Macro results

* Allows to pass the macro result to the clients (string representation) and to the wrapper macro
* Results are defined as the `result_def` class member of the macro class
* A result is characterized by: name, type, default value and description - exactly the same as macro parameter
* Multiple results are possible

### Macro results - demo

* `prdef twice`
* `twice 2`

### Macro data

* Allows to pass the macro data to the clients (serialized with `pickle`) and to the wrapper macro
* Data format is totally opened

### Macro data - demo

* Execute macro twice: `twice 3`
* Access macro data in Spock with: `macrodata`
* Execute scan: `ascan mot01 0 10 2 0.1`
* Access macro data with in Spock with: `macrodata`

### Sequencing and nesting macros

* Programmatically
  * Macros can call other macros, and these other macros can call another macros, and so on...
  * The easiest way to program execution of a macro from wihin another macro is to use the macro context: `edmac hello_world` and add `self.lsm()`

### Sequencing and nesting macros

* Graphically using _sequencer_:
  * Open `demogui` and show `Sequencer` panel
  * Add macros to the sequence e.g. `lsm`, `ct` and `ascan` and fill their parameters
  * Start the sequence execution, stop it, pause it, etc..
  * Save the sequence to a file and open it back

### Hooks

* Python code that will be executed at given points of the macro
* Can be a Python callable or a macro
* Hooks can be attached to the hook places that defines macros
* Macro must inherit from `Hookable` class to allow hook places
* Hook places are defined and handled by the macro in the code
* Exists three possibilities to attach hooks to the macros:
 * configuring general hooks - using the `lsgh`, `defgh`, `udefgh`.
 * programatically - using the `Hookable` class API (it requires defining a wrapper macro)
 * graphically - using the `sequencer` widget


### Hooks - demo

* `prdef loop`
* `prdef captain_hook`
* `captain_hook 0 10 1`

### General Hooks - demo

* `lsgh`
* `defgh wa pre-acq`
* `ct`
* `udefgh wa`
* `ct`

### Interactive macros

* Macros can ask for user input
* Macro must be declared as interactive e.g. `imacro` decorator or `interactive` class member set to `True`
* The user input request may be characterized with: type, default value, title, unit, label, range, ...
* Spock may handle user input request either in the CLI or GUI mode - see `SPOCK_INPUT_HANDLER` in the `sardana.sardanacustomsettings`

### Interactive macros - demo

* `prdef ask_number_of_points`
* `ask_number_of_points`
* `prdef ask_for_moveable`
* `ask_for_moveable`

### Ploting in macros

* Macro send request to plot to the client e.g. spock
* Uses `matplotlib.pyplot` framework API e.g. `pyplot.plot`

### Ploting in macros - demo

* `prdef random_image`
* `random_image`

### Interrupting macro execution

* Macros  can be stopped, aborted or paused
* As soon as the user requested the macro interruption the macro will get interrupted on the next call of `checkPoint` (or `pausePoint`) or any of the macro API method (Macro class methods decorated with `mAPI`)
* Long sleeps should be interleaved with check points
* Macro may define `on_stop`, `on_abort` and `on_pause` method that will be called on the corresponding interruption
* Macro reserved object, for example requested with the `getMotion` method (`getMotor` will not reserve the motor object!), are stopped/aborted on the corresponding interruption (before calling the `on_stop`/`on_abort` method)

### Reporting macro progress

* Macros may report their progress while are executed
* The progress is usually expressed in percentage 0 - 100 %
* The progress update is done using the `yield` statement within the `run` method

### Macro environment

* Key-value store accessible from macros and clients
* Stored persistently
* API: `getEnv` and `setEnv` or simply `senv` macro
* Different levels: Global, Door, Macro
* Example:
  * `senv ActiveMntGrp mntgrp01`
  * `ct`

# MACROBUTTON

## A GUI button to execute a single macro

### MACROBUTTON: A GUI button to execute a single macro
  
  
* **macrobutton** is a Widget button allowing execution of a single macro
* It can be integraged in Qt GUIs
* It allows: 
    * Running macro: by clicking once 
    * Interrupting a macro: by clicking again
        * It is possible to abort the macro
        * It is possible to continue with the execution
    * To see the macro progress

### MACROBUTTON: A GUI button to execute a single macro -> DEMO
  

**Demo** of a macrobutton executed on the container:

Run spock to see the results:  
```
spock --profile demo1
```  

Execute the **macrobutton** code, which in this case **executes the 'lsm' macro**, and see the **output of the executed macro in spock**.

* macrobutton executing an lsm macro:  
```
python3 ~/sardana-training/res/macrobutton/macrobutton_lsm.py
```

* macrobutton executing an ascan macro:  
```
python3 ~/sardana-training/res/macrobutton/macrobutton_scan.py
```  

  

### MACROBUTTON: BL13-Xaloc ALBA Beamline Use Case

BL13-Xaloc Beamline uses Macrobuttons in a TaurusGUI for focusing the sample image and for looking for the best diffraction position on the sample

![BL13-Xaloc GUI with Macrobuttons](res/OAV_raster.jpg)

  